In [8]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/home/ubuntu/varios/skforecast'

In [21]:
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregMultiSeriesCustom import ForecasterAutoregMultiSeriesCustom
import numpy as np
from sklearn.linear_model import LinearRegression
import joblib
import inspect

In [10]:
# Custom functions
def custom_pred(y):
    return np.mean(y)

def custom_weights(y):
    return np.ones(len(y))

def custom_weights2(y):
    return np.arange(1, len(y)+1)

In [23]:
# forecaster = ForecasterAutoregCustom(
#     regressor = LinearRegression(),
#     window_size=50,
#     fun_predictors= custom_pred,
#     weight_func= custom_weights
# )


forecaster = ForecasterAutoregMultiSeriesCustom(
    regressor = LinearRegression(),
    window_size=50,
    fun_predictors= custom_pred,
    weight_func= {'series 1': custom_weights, 'series 2': custom_weights2}
)

In [24]:
def save_forecaster(
    forecaster, 
    file_name: str,
    save_custom_functions: bool=True, 
    verbose: bool=True
) -> None:
    """
    Save forecaster model using joblib. If custom functions are used to create
    predictors or weights, they are saved as .py files.

    Parameters
    ----------
    forecaster: forecaster
        Forecaster created with skforecast library.
    file_name: str
        File name given to the object.
    save_custom_functions: bool, default `True`
        If True, save custom functions used in the forecaster (fun_predictors and
        weight_func) as .py files. Custom functions need to be available in the
        environment where the forecaster is going to be loaded.
    verbose: bool, default `True`
        Print summary about the forecaster saved.

    Returns
    -------
    None

    """

    # Save forecaster
    joblib.dump(forecaster, filename=file_name)

    # Save custom functions to create predictors
    if hasattr(forecaster, 'fun_predictors') and forecaster.fun_predictors is not None:
        file_name = forecaster.fun_predictors.__name__ + '.py'
        with open(file_name, 'w') as file:
            file.write(inspect.getsource(forecaster.fun_predictors))

    # Save custom functions to create weights
    if hasattr(forecaster, 'weight_func') and forecaster.weight_func is not None:
        if isinstance(forecaster.weight_func, dict):
            for fun in set(forecaster.weight_func.values()):
                file_name = fun.__name__ + '.py'
                with open(file_name, 'w') as file:
                    file.write(inspect.getsource(fun))
        else:
            file_name = forecaster.weight_func.__name__ + '.py'
            with open(file_name, 'w') as file:
                file.write(inspect.getsource(forecaster.weight_func))

    if verbose:
        forecaster.summary()

In [25]:
save_forecaster(forecaster, 'forecaster.joblib')

ForecasterAutoregMultiSeriesCustom 
Regressor: LinearRegression() 
Predictors created with function: custom_pred 
Transformer for series: None 
Transformer for exog: None 
Window size: 50 
Series levels (names): None 
Series weights: None 
Weight function included: True 
Exogenous included: False 
Type of exogenous variable: None 
Exogenous variables names: None 
Training range: None 
Training index type: None 
Training index frequency: None 
Regressor parameters: {'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'positive': False} 
fit_kwargs: {} 
Creation date: 2023-12-07 11:10:30 
Last fit date: None 
Skforecast version: 0.11.0 
Python version: 3.11.4 
Forecaster id: None 

